In [ ]:
# ensure that you have jupyter installed in your environment!

import pandas as pd
import openap
from traffic.core import Flight

# if you have not added the gedai folder to your PATH, then we need to append 
# the package using sys
import sys
sys.path.append("../")
import gedai

The first step is to load the raw ADS-B data.
For this example, we are going to follow the DLR's HALO (D-ADLR) Gulfstream 5 research aircraft.
The HALO has the unique ICAO transponder hex `3c5192`.
We will use the private-jets database compiled by Gössling et al. (2024) on 06/08/2022.

Note that ADS-B Exchange provides a free API for the first day of the month, so you can use the data here to do the same analysis.

In [ ]:
icao = "3c5192"

# option 1: with ADS-B Exchange
# base_url = "https://samples.adsbexchange.com/traces/2022/12/01/"
# source = "adsb_exchange"
# data, metadata = gedai.fetch_raw_data(source, base_url, icao)

# option 2: with bjets database
base_url = "https://private-jets.fra1.digitaloceanspaces.com/globe_history/2022-08-06/"
source = "adsb_exchange"
data, metadata = gedai.fetch_raw_data("bjets", base_url, icao)

data

Fetching was successful!
We can now move on to creating the pandas DataFrame and Flight object.
We will use openap's aircraft class for this, for which a .yml file must be available within your OpenAP installation.
Let's first check which aircraft are supported.

In [ ]:
available_aircraft = openap.prop.available_aircraft()
print(f"Supports {len(available_aircraft)} aircraft types")
print(available_aircraft)

OpenAP supports the `glf6` type (Gulfstream G650), which is close enough to the `glf5` (Gulfstream 5) that we are after.
Let's now create this aircraft class.

Note: If you have access to BADA3, you can also generate an aircraft object using:

```python
from openap.addon import bada3
bada_path = "/path/to/bada3/files/"
ac = bada3.bada3_aircraft(metadata["t"], bada_path)
```

In [ ]:
ac_type = "GLF6"
ac = openap.prop.aircraft(ac_type)
ac

Now let's pre-process the data and create a Flight instance.
We will use this to perform filtering etc.
GEDAI is built on top of the `traffic` package, so we can view the Flight directly, giving us an insight into the flight duration and sampling rate.

In [ ]:
df = gedai.create_dataframe(data, metadata, source)
f = Flight(df)

This is clearly a single flight, so there is no need to split the flight into legs.
For reference, this can be done using `gedai.split_by_leg(f, source="adsb_exchange")`.

Let's use some plots to view the flight in more detail.

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# plot data
ax = plt.axes(projection=ccrs.PlateCarree())
ax.plot(f.data.longitude, f.data.latitude, "r", transform=ccrs.Geodetic())
ax.coastlines()

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].scatter(f.data.timestamp, f.data.altitude)
ax[1].scatter(f.data.timestamp, f.data.groundspeed)

The flight crossed the Atlantic and ADS-B data is thus missing!
This is unfortunately a common problem, but there are ways of salvaging the situation.
What we can do is filter and then resample the data, using a great circle distance to create points where we don't have any.
This is not perfect, but it's something.

In [ ]:
g = f.phases().filter().resample("20s")

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].scatter(g.data.timestamp, g.data.altitude)
ax[1].scatter(g.data.timestamp, g.data.groundspeed)

Okay, it's clearly not perfect, but it will do for now.

Let's now calculate the fuel flow.
Note: the `compute_fuel_flow` function currently only works for the BADA3 add-on, so we will do this manually.

In [ ]:
g = gedai.calculate_distance(g)
m_0 = ac["mtow"] * 0.9
fuelflow = openap.FuelFlow(ac_type)

# the compute_fuel_flow function currently only works for BADA3
# g = gedai.compute_fuel_flow(g, fuelflow, m_0, ac)

# calculate fuel flow manually iteratively
ff_lst = []
fuel_lst = []
mass_current = m_0

d_ts = g.data.timestamp.diff().dt.total_seconds().bfill().values
for i, row in g.data.iterrows():
    ff = fuelflow.enroute(
        mass=mass_current,
        tas=row.groundspeed,
        alt=row.altitude,
        vs=row.vertical_rate,
    )
    if row.phase == "GROUND" or row.phase == "NA":
        ff = 0.0
    fuel = ff * d_ts[i]
    ff_lst.append(ff)
    fuel_lst.append(ff * d_ts[i])
    mass_current -= fuel
g = g.assign(fuelflow=ff_lst, fuel=fuel_lst)

g.data

Now we can look at the emissions.

In [ ]:
# calculate emissions
g = gedai.calc_emissions(g, ac)

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].scatter(g.data.timestamp, g.data.co2flow, label="CO2 [kg/s]")
ax[0].scatter(g.data.timestamp, g.data.h2oflow, label="H2O [kg/s]")
ax[1].scatter(g.data.timestamp, g.data.noxflow, label="NOx [kg/s]")

ax[0].legend(loc="best")
ax[1].legend(loc="best")